In [ ]:
!pip install huggingface
!pip install transformers[torch]
!pip install datasets
!pip install evaluate
!pip install seqeval
!pip install accelerate -U

In [ ]:
import huggingface
import huggingface_hub
from datasets import load_dataset
import datasets
import numpy as np
import torch

from transformers import AutoTokenizer
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

import evaluate

In [ ]:
username = "HUGGINFACE_USERNAME
model1 = "RISE_NER"
model2 = "RISE_NER_REDUCED"

In [ ]:
key = "HUGGINGFACE_KEY"
huggingface_hub.login(key)

In [ ]:
train_dataset = load_dataset("Babelscape/multinerd", split="train")
valid_dataset = load_dataset("Babelscape/multinerd", split="validation")
test_dataset  = load_dataset("Babelscape/multinerd", split="test")

In [ ]:
train_en = train_dataset.filter(lambda x: x["lang"] == 'en')
valid_en = valid_dataset.filter(lambda x: x["lang"] == 'en')
test_en = test_dataset.filter(lambda x: x["lang"] == 'en')

In [ ]:
train_en[0]

In [ ]:
label2id = {
    "O": 0,
    "B-PER": 1,
    "I-PER": 2,
    "B-ORG": 3,
    "I-ORG": 4,
    "B-LOC": 5,
    "I-LOC": 6,
    "B-ANIM": 7,
    "I-ANIM": 8,
    "B-BIO": 9,
    "I-BIO": 10,
    "B-CEL": 11,
    "I-CEL": 12,
    "B-DIS": 13,
    "I-DIS": 14,
    "B-EVE": 15,
    "I-EVE": 16,
    "B-FOOD": 17,
    "I-FOOD": 18,
    "B-INST": 19,
    "I-INST": 20,
    "B-MEDIA": 21,
    "I-MEDIA": 22,
    "B-MYTH": 23,
    "I-MYTH": 24,
    "B-PLANT": 25,
    "I-PLANT": 26,
    "B-TIME": 27,
    "I-TIME": 28,
    "B-VEHI": 29,
    "I-VEHI": 30,
  }
id2label = {value: key for key, value in label2id.items()}

wanted_tags = ['B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-ANIM', 'I-ANIM', 'B-DIS', 'I-DIS', "O"]

In [ ]:
def is_in(key):
  """Checks if given key is one of the 5 categories defined by the task"""
  return id2label[key] in wanted_tags

def test_is_in():
  assert (is_in(1) == True)
  assert (is_in(20) == False)
  assert (is_in(2) == True)
  assert (is_in(3) == True)
  assert (is_in(4) == True)
  assert (is_in(5) == True)
  assert (is_in(6) == True)
  assert (is_in(7) == True)
  assert (is_in(8) == True)
  assert (is_in(13) == True)
  assert (is_in(14) == True)
  print('Test complete. No Errors')

test_is_in()

def convert_to_five_categories(key):
  """Takes a tag and either keeps it as is or converts it to 0 if not one of the desired categories"""
  if is_in(key):
    return key
  else:
    return 0

def test_converter():
  assert (convert_to_five_categories(1) == 1)
  assert (convert_to_five_categories(20) == 0)
  assert (convert_to_five_categories(2) == 2)
  assert (convert_to_five_categories(3) == 3)
  assert (convert_to_five_categories(4) == 4)
  assert (convert_to_five_categories(5) == 5)
  assert (convert_to_five_categories(6) == 6)
  assert (convert_to_five_categories(7) == 7)
  assert (convert_to_five_categories(8) == 8)
  assert (convert_to_five_categories(13) == 13)
  assert (convert_to_five_categories(14) == 14)
  print('Test complete. No Errors')
test_converter()


def convert_vector(v):
  """Reduces a vector of labels down to either 0 or one of the 5 desired categories """
  return [convert_to_five_categories(k) for k in v]

def test_convert_vector():
  assert((convert_vector([0, 0, 0, 0, 5, 0]) == [0, 0, 0, 0, 5, 0]))
  assert((convert_vector([0, 0, 0, 0, 5, 6]) == [0, 0, 0, 0, 5, 6]))
  assert((convert_vector([0, 0, 0, 0, 17, 0]) == [0, 0, 0, 0, 0, 0]))
  assert((convert_vector([0, 0, 0, 0, 1, 0]) == [0, 0, 0, 0, 1, 0]))
  print('Test complete. No Errors')
test_convert_vector()

def convert_datapoint(example):
  example['ner_tags'] = convert_vector(example['ner_tags'])
  return example


In [ ]:
train_reduced = train_en.map(convert_datapoint)

In [ ]:
valid_reduced = valid_en.map(convert_datapoint)
test_reduced = test_en.map(convert_datapoint)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
tokenizer(train_en[0]['tokens'], truncation=True, is_split_into_words=True, return_tensors="pt", padding=True)

In [ ]:
def tokenize_and_align_labels(examples):
  """Re-align the NER tags after sub-word tokenization
  CREDIT: https://huggingface.co/docs/transformers/tasks/token_classification"""
  tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True, return_tensors="pt", padding=True)

  labels = []
  for i, label in enumerate(examples[f"ner_tags"]):
    word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:  # Set the special tokens [CLS] and [SEP] to -100.
      if word_idx is None:
        label_ids.append(-100)
      elif word_idx != previous_word_idx:  # Only label the first token of a given word.
        label_ids.append(label[word_idx])
      else:
        label_ids.append(-100)
      previous_word_idx = word_idx
    labels.append(label_ids)

  tokenized_inputs["labels"] = torch.tensor(labels)
  return tokenized_inputs

In [ ]:
tokenize_and_align_labels(train_en[0:2])

In [ ]:
tokenized_train =  train_en.map(tokenize_and_align_labels, batched=True)
tokenized_valid = valid_en.map(tokenize_and_align_labels, batched=True)
tokenized_test = test_en.map(tokenize_and_align_labels, batched=True)

tokenized_train_reduced =  train_reduced.map(tokenize_and_align_labels, batched=True)
tokenized_valid_reduced = valid_reduced.map(tokenize_and_align_labels, batched=True)
# tokenized_test_reduced = test_reduced.map(tokenize_and_align_labels, batched=True)


In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
seqeval = evaluate.load("seqeval")

In [ ]:

def compute_metrics(p):
    """First re-aligns the labels to the original state and then calculates metrics
    """
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
label_list = list(label2id.keys())

Train the models using the traininer class

In [ ]:

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=31, id2label=id2label, label2id=label2id
)


training_args = TrainingArguments(
    output_dir="RISE_NER",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.push_to_hub()

In [ ]:

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=31, id2label=id2label, label2id=label2id
)


training_args = TrainingArguments(
    output_dir="RISE_NER_REDUCED",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_reduced,
    eval_dataset=tokenized_valid_reduced,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.push_to_hub()


This is a repetition of the trainer code but loading the model from hugginface_hub. We do this to use the predict function of the trainer to evaluate on the test set.

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(f"{username}/RISE_NER")


training_args = TrainingArguments(
    output_dir="RISE_NER",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

model.eval()

predictions = trainer.predict(tokenized_test)

In [ ]:
predictions

In [ ]:
model2 = AutoModelForTokenClassification.from_pretrained(f"{username}/RISE_NER_REDUCED")


training_args = TrainingArguments(
    output_dir="RISE_NER_REDUCED",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer2 = Trainer(
    model=model2,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

model2.eval()

predictions = trainer2.predict(tokenized_test)

In [ ]:
predictions